In [6]:
import os
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [10]:
os.environ.clear()
load_dotenv("../../.env", override=True) 
print(os.getenv("ANTHROPIC_API_KEY"))

123


In [18]:
from pydantic import BaseModel, Field
from typing import List, Optional
from anthropic import AnthropicVertex
import json

# Example complex model structure
class Batsman(BaseModel):
    name: str
    
class Bowler(BaseModel):
    name: str
    
class WicketKeeper(BaseModel):
    name: str
    
class AgentResponse(BaseModel):
    batsmen: List[Batsman]
    bowlers: List[Bowler]
    wicketkeeper: List[WicketKeeper]
    
def format_model_schema(model_class: type[BaseModel]) -> str:
    """Convert Pydantic model to a prompt-friendly schema description"""
    schema = model_class.model_json_schema()
    return json.dumps(schema, indent=2)

class VertexStructuredAgent:
    def __init__(self, location: str, project_id: str):
        self.client = AnthropicVertex(
            region=location,
            project_id=project_id
        )
        
    def get_structured_response(self, 
                              user_input: str, 
                              response_model: type[BaseModel]) -> BaseModel:
        """Get a response from Claude that conforms to the given Pydantic model"""
        
        prompt = f"""Please provide your response in JSON format that exactly matches this schema:
        
        {format_model_schema(response_model)}
        
        The JSON must be valid and match all types and constraints.
        
        User input: {user_input}"""
        
        response = self.client.messages.create(
            max_tokens=1024,
            messages=[{
                "role": "user",
                "content": prompt
            }],
            model="claude-3-5-sonnet-v2@20241022"
        )
        
        try:
            print(response.content[0].text)
            # Extract JSON from response (handles cases where Claude might add explanatory text)
            json_str = self._extract_json(response.content[0].text)
            # Parse and validate against the model
            return response_model.model_validate_json(json_str)
        except Exception as e:
            raise ValueError(f"Failed to parse response into expected model: {str(e)}")
            
    def _extract_json(self, text: str) -> str:
        """Extract JSON object from text that might contain other content"""
        try:
            # Find JSON-like content between curly braces
            start = text.find('{')
            end = text.rfind('}') + 1
            if start >= 0 and end > 0:
                possible_json = text[start:end]
                # Validate it's actually JSON
                json.loads(possible_json)
                return possible_json
            raise ValueError("No JSON object found in response")
        except json.JSONDecodeError:
            raise ValueError("Invalid JSON in response")

# Example usage
agent = VertexStructuredAgent(
    location="us-east5",
    project_id="hackathon-2025-450908"
)

try:
    response = agent.get_structured_response(
        "Tell the top 3 Indian cricket players of each category of all time",
        AgentResponse
    )
    
    # Now you can access the structured data
    print("Thoughts:", [t.name for t in response.batsmen])
    if response.bowler:
        print("Tools to call:", [t.name for t in response.bowler])
        
except ValueError as e:
    print(f"Error getting structured response: {e}")

/Users/sanchitgupta/Desktop/repos/data-mapper-ai/venv/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


{
  "batsmen": [
    {
      "name": "Sachin Tendulkar"
    },
    {
      "name": "Virat Kohli"
    },
    {
      "name": "Sunil Gavaskar"
    }
  ],
  "bowlers": [
    {
      "name": "Anil Kumble"
    },
    {
      "name": "Kapil Dev"
    },
    {
      "name": "Ravichandran Ashwin"
    }
  ],
  "wicketkeeper": [
    {
      "name": "MS Dhoni"
    },
    {
      "name": "Syed Kirmani"
    },
    {
      "name": "Nayan Mongia"
    }
  ]
}
Thoughts: ['Sachin Tendulkar', 'Virat Kohli', 'Sunil Gavaskar']


AttributeError: 'AgentResponse' object has no attribute 'bowler'